In [ ]:
import datetime
import random
import dataclasses
from pathlib import Path

from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import numpy as np
import physiokit as pk


In [ ]:
plotly_template = "plotly_dark"
bg_color = "rgba(38,42,50,1.0)"
primary_color = "#11acd5" # "rgb(101, 110, 242)"
secondary_color = "#ce6cff"
tertiary_color = "rgb(234,52,36)"
quaternary_color = "rgb(34,15,88)"

pio.templates.default = plotly_template


In [ ]:
dst_path = Path("../docs/assets")
fs = 1000 # Hz


In [ ]:
tgt_hr = 64 # BPM

# Generate synthetic ECG signal
ecg = pk.ecg.synthesize(duration=8, sample_rate=fs, heart_rate=tgt_hr, leads=1)

# Add noise
ecg_noise = pk.signal.add_baseline_wander(ecg, amplitude=2, frequency=1, sample_rate=fs)
ecg_noise = pk.signal.add_powerline_noise(ecg_noise, amplitude=0.05, frequency=60, sample_rate=fs)
ecg_noise = pk.signal.add_noise_sources(ecg_noise, amplitudes=[0.05, 0.05], frequencies=[60, 80], sample_rate=fs)

# Create timestamps
tod = datetime.datetime(2025, 5, 24, random.randint(12, 23), 00)
ts = np.arange(0, ecg.size) / fs

# Clean ECG signal
ecg_clean = pk.ecg.clean(ecg_noise, lowcut=2, highcut=30, order=5, sample_rate=fs)

# Compute heart rate
hr_bpm, _ = pk.ecg.compute_heart_rate(ecg_clean, sample_rate=fs)

# Extract R-peaks and RR-intervals
peaks = pk.ecg.find_peaks(ecg_clean, sample_rate=fs)
rri = pk.ecg.compute_rr_intervals(peaks)
mask = pk.ecg.filter_rr_intervals(rri, sample_rate=fs)

# Re-compute heart rate
hr_bpm = 60 / (np.nanmean(rri[mask == 0]) / fs)

# Compute HRV metrics
hrv_td = pk.hrv.compute_hrv_time(rri[mask == 0], sample_rate=fs)

bands = [(0.04, 0.15), (0.15, 0.4), (0.4, 0.5)]
hrv_fd = pk.hrv.compute_hrv_frequency(peaks[mask == 0], rri[mask == 0], bands=bands, sample_rate=fs)


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ts,
    y=ecg, name='ECG',
    line_width=2,
    line_color=primary_color,
    mode="lines"
))
fig.update_yaxes(title='Amplitude (mV)')
fig.update_xaxes(title='Time (s)')
fig.update_layout(
    template="plotly_dark",
    plot_bgcolor=bg_color,
    paper_bgcolor=bg_color,
    margin=dict(l=10, r=5, t=20, b=20),
    height=300,
)
fig.write_html(dst_path / f"pk-synthetic-ecg-raw.html", include_plotlyjs='cdn', full_html=False)
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ts,
    y=ecg_noise, name='ECG',
    line_color=primary_color,
    line_width=2,
    mode="lines"
))
fig.update_yaxes(title='Amplitude (mV)')
fig.update_xaxes(title='Time (s)')
fig.update_layout(
    template="plotly_dark",
    plot_bgcolor=bg_color,
    paper_bgcolor=bg_color,
    margin=dict(l=10, r=5, t=20, b=20),
    height=300,
)
fig.write_html(dst_path / f"pk-synthetic-ecg-noise.html", include_plotlyjs='cdn', full_html=False)
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ts,
    y=ecg_noise, name='Raw ECG',
    line_color=primary_color,
    line_width=2,
    mode="lines"
))
fig.add_trace(go.Scatter(
    x=ts,
    y=ecg_clean,
    line_color=secondary_color,
    name='Clean ECG',
    line_width=2,
))
fig.update_yaxes(title='Amplitude (mV)')
fig.update_xaxes(title='Time (s)')
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    template="plotly_dark",
    plot_bgcolor=bg_color,
    paper_bgcolor=bg_color,
    margin=dict(l=10, r=5, t=60, b=20),
    height=300,
)
fig.write_html(dst_path / f"pk-synthetic-ecg-clean.html", include_plotlyjs='cdn', full_html=False)

fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ts,
    y=ecg_clean,
    line_color=secondary_color,
    name='ECG',
    line_width=2,
))
for peak in peaks:
    fig.add_vline(
        x=ts[peak],
        line_width=1,
        line_dash="dash",
        line_color="white",
        annotation={"text": "R-Peak", "textangle": -90}
    )
fig.update_yaxes(title='Amplitude (mV)')
fig.update_xaxes(title='Time (s)')
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    template="plotly_dark",
    plot_bgcolor=bg_color,
    paper_bgcolor=bg_color,
    margin=dict(l=10, r=5, t=60, b=20),
    height=300,
)
fig.write_html(dst_path / f"pk-synthetic-ecg-rpeak.html", include_plotlyjs='cdn', full_html=False)
fig.show()


In [ ]:
# Generate synthetic PPG signal
ppg = pk.ppg.synthesize(
    duration=10,
    sample_rate=fs,
    heart_rate=tgt_hr
)[:8*fs]

# Add baseline wander
ppg_noise = pk.signal.add_baseline_wander(
    data=ppg,
    amplitude=2,
    frequency=1,
    sample_rate=fs
)

# Add powerline noise
ppg_noise = pk.signal.add_powerline_noise(
    data=ppg_noise,
    amplitude=0.05,
    frequency=60,
    sample_rate=fs
)

# Add additional noise sources
ppg_noise = pk.signal.add_noise_sources(
    data=ppg_noise,
    amplitudes=[0.05, 0.05],
    frequencies=[10, 20],
    sample_rate=fs
)

# Create timestamps
tod = datetime.datetime(2025, 5, 24, random.randint(12, 23), 00)
ts = np.arange(0, ppg.size) / fs

# Clean PPG signal
ppg_clean = pk.ppg.clean(
    data=ppg_noise,
    lowcut=0.5,
    highcut=4,
    order=3,
    sample_rate=fs
)


# Extract s-peaks and peak-to-peak intervals
peaks = pk.ppg.find_peaks(data=ppg_clean, sample_rate=fs)

# Compute RR-intervals
rri = pk.ppg.compute_rr_intervals(peaks=peaks)

# Filter RR-intervals
mask = pk.ppg.filter_rr_intervals(rr_ints=rri, sample_rate=fs)

# Compute heart rate
hr_bpm, _ = pk.ppg.compute_heart_rate(
    data=ppg_clean,
    method="fft",
    sample_rate=fs
)

# Re-compute heart rate
hr_bpm = 60 / (np.nanmean(rri[mask == 0]) / fs)

# Compute HRV metrics
hrv_td = pk.hrv.compute_hrv_time(rri[mask == 0], sample_rate=fs)

bands = [(0.04, 0.15), (0.15, 0.4), (0.4, 0.5)]
hrv_fd = pk.hrv.compute_hrv_frequency(peaks[mask == 0], rri[mask == 0], bands=bands, sample_rate=fs)


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ts,
    y=ppg,
    name='PPG',
    line_width=2,
    line_color=primary_color,
    mode="lines"
))
fig.update_yaxes(title='Amplitude (mV)')
fig.update_xaxes(title='Time (s)')
fig.update_layout(
    template="plotly_dark",
    plot_bgcolor=bg_color,
    paper_bgcolor=bg_color,
    margin=dict(l=10, r=5, t=20, b=20),
    height=300,
)
fig.write_html(dst_path / f"pk-synthetic-ppg-raw.html", include_plotlyjs='cdn', full_html=False)
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ts,
    y=ppg_noise,
    name='PPG',
    line_color=primary_color,
    line_width=2,
    mode="lines"
))
fig.update_yaxes(title='Amplitude (mV)')
fig.update_xaxes(title='Time (s)')
fig.update_layout(
    template="plotly_dark",
    plot_bgcolor=bg_color,
    paper_bgcolor=bg_color,
    margin=dict(l=10, r=5, t=20, b=20),
    height=300,
)
fig.write_html(dst_path / f"pk-synthetic-ppg-noise.html", include_plotlyjs='cdn', full_html=False)
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ts,
    y=ppg_noise,
    name='Raw PPG',
    line_color=primary_color,
    line_width=2,
    mode="lines"
))
fig.add_trace(go.Scatter(
    x=ts,
    y=ppg_clean,
    line_color=secondary_color,
    name='Clean PPG',
    line_width=2,
))
fig.update_yaxes(title='Amplitude (mV)')
fig.update_xaxes(title='Time (s)')
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    template="plotly_dark",
    plot_bgcolor=bg_color,
    paper_bgcolor=bg_color,
    margin=dict(l=10, r=5, t=60, b=20),
    height=300,
)
fig.write_html(dst_path / f"pk-synthetic-ppg-clean.html", include_plotlyjs='cdn', full_html=False)

fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ts,
    y=ppg_clean,
    line_color=secondary_color,
    name='PPG',
    line_width=2,
))
for peak in peaks:
    fig.add_vline(
        x=ts[peak],
        line_width=1,
        line_dash="dash",
        line_color="white",
        annotation={"text": "S-Peak", "textangle": -90}
    )
fig.update_yaxes(title='Amplitude (mV)')
fig.update_xaxes(title='Time (s)')
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    template="plotly_dark",
    plot_bgcolor=bg_color,
    paper_bgcolor=bg_color,
    margin=dict(l=10, r=5, t=60, b=20),
    height=300,
)
fig.write_html(dst_path / f"pk-synthetic-ppg-rpeak.html", include_plotlyjs='cdn', full_html=False)
fig.show()


In [ ]:
ppg_red = ...
ppg_ir = ...
max8614x_coefs = [-16.666666, 8.333333, 100]

# Compute SpO2 in time domain
spo2_td = pk.ppg.compute_spo2_in_time(
    ppg1=ppg_red,
    ppg2=ppg_ir,
    coefs=max8614x_coefs,
    lowcut=0.5,
    highcut=4,
    sample_rate=fs
)

# Compute SpO2 in frequency domain
spo2_fd = pk.ppg.compute_spo2_in_frequency(
    ppg1=ppg_red,
    ppg2=ppg_ir,
    coefs=max8614x_coefs,
    lowcut=0.5,
    highcut=4,
    sample_rate=fs
)


In [ ]:
tgt_rr = 22 # BPM

# Generate synthetic PPG signal
rsp = pk.rsp.synthesize(
    duration=60,
    sample_rate=fs,
    respiratory_rate=tgt_rr,
)

# Add baseline wander
rsp_noise = pk.signal.add_baseline_wander(
    data=rsp,
    amplitude=2,
    frequency=.05,
    sample_rate=fs
)

# Add powerline noise
rsp_noise = pk.signal.add_powerline_noise(
    data=rsp_noise,
    amplitude=0.05,
    frequency=60,
    sample_rate=fs
)

# Add additional noise sources
rsp_noise = pk.signal.add_noise_sources(
    data=rsp_noise,
    amplitudes=[0.05, 0.05],
    frequencies=[10, 20],
    sample_rate=fs
)

# Create timestamps
tod = datetime.datetime(2025, 5, 24, random.randint(12, 23), 00)
ts = np.arange(0, rsp.size) / fs

# Clean RSP signal
rsp_clean = pk.rsp.clean(
    data=rsp_noise,
    lowcut=0.05,
    highcut=3,
    order=3,
    sample_rate=fs
)


# Extract respiratory cycles
peaks = pk.rsp.find_peaks(data=rsp_clean, sample_rate=fs)

# Compute RR-intervals
rri = pk.rsp.compute_rr_intervals(peaks=peaks)

# Filter RR-intervals
mask = pk.rsp.filter_rr_intervals(rr_ints=rri, sample_rate=fs)

# Compute respiratory rate
rr_bpm, rr_qos = pk.rsp.compute_respiratory_rate(
    data=rsp_clean,
    method="fft",
    sample_rate=fs,
    lowcut=0.05,
    highcut=1
)
print(rr_bpm, rr_qos)

# # Re-compute respiratory rate
hr_bpm = 60 / (np.nanmean(rri[mask == 0]) / fs)
print(hr_bpm)


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ts,
    y=rsp,
    name='RSP',
    line_width=2,
    line_color=primary_color,
    mode="lines"
))
fig.update_yaxes(title='Amplitude (mV)')
fig.update_xaxes(title='Time (s)')
fig.update_layout(
    template="plotly_dark",
    plot_bgcolor=bg_color,
    paper_bgcolor=bg_color,
    margin=dict(l=10, r=5, t=20, b=20),
    height=300,
)
fig.write_html(dst_path / f"pk-synthetic-rsp-raw.html", include_plotlyjs='cdn', full_html=False)
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ts,
    y=rsp_noise,
    name='RSP',
    line_color=primary_color,
    line_width=2,
    mode="lines"
))
fig.update_yaxes(title='Amplitude (mV)')
fig.update_xaxes(title='Time (s)')
fig.update_layout(
    template="plotly_dark",
    plot_bgcolor=bg_color,
    paper_bgcolor=bg_color,
    margin=dict(l=10, r=5, t=20, b=20),
    height=300,
)
fig.write_html(dst_path / f"pk-synthetic-rsp-noise.html", include_plotlyjs='cdn', full_html=False)
fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ts,
    y=rsp_noise,
    name='Raw RSP',
    line_color=primary_color,
    line_width=2,
    mode="lines"
))
fig.add_trace(go.Scatter(
    x=ts,
    y=rsp_clean,
    line_color=secondary_color,
    name='Clean RSP',
    line_width=2,
))
fig.update_yaxes(title='Amplitude (mV)')
fig.update_xaxes(title='Time (s)')
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    template="plotly_dark",
    plot_bgcolor=bg_color,
    paper_bgcolor=bg_color,
    margin=dict(l=10, r=5, t=60, b=20),
    height=300,
)
fig.write_html(dst_path / f"pk-synthetic-rsp-clean.html", include_plotlyjs='cdn', full_html=False)

fig.show()


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=ts,
    y=rsp_clean,
    line_color=secondary_color,
    name='RSP',
    line_width=2,
))
for peak in peaks:
    fig.add_vline(
        x=ts[peak],
        line_width=1,
        line_dash="dash",
        line_color="white",
        # annotation={"text": "R-Peak", "textangle": -90}
    )
fig.update_yaxes(title='Amplitude (mV)')
fig.update_xaxes(title='Time (s)')
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    template="plotly_dark",
    plot_bgcolor=bg_color,
    paper_bgcolor=bg_color,
    margin=dict(l=10, r=5, t=20, b=20),
    height=300,
)
fig.write_html(dst_path / f"pk-synthetic-rsp-rpeak.html", include_plotlyjs='cdn', full_html=False)
fig.show()


In [ ]:
# Read ribcage and abdominal band data
rsp_rc = ...
rsp_ab = ...

# Compute dual band metrics
pk.rsp.compute_dual_band_metrics(
    rc=rsp_rc,
    ab=rsp_ab,
    sample_rate=fs,
    lowcut=0.05,
    highcut=1,
    order=3
)


In [ ]:
tgt_hr = 64 # BPM

# Generate synthetic ECG signal
ecg = pk.ecg.synthesize(
    duration=8,
    sample_rate=fs,
    heart_rate=tgt_hr,
    leads=1
)

# Create timestamps
tod = datetime.datetime(2025, 5, 24, random.randint(12, 23), 00)
ts = np.arange(0, ecg.size) / fs

# Add noise
ecg_noise = pk.signal.add_baseline_wander(ecg, amplitude=2, frequency=1, sample_rate=fs)
ecg_noise = pk.signal.add_powerline_noise(ecg_noise, amplitude=0.05, frequency=60, sample_rate=fs)
ecg_noise = pk.signal.add_noise_sources(ecg_noise, amplitudes=[0.05, 0.05], frequencies=[60, 80], sample_rate=fs)

# Clean ECG signal
ecg_clean = pk.ecg.clean(
    data=ecg_noise,
    lowcut=2,
    highcut=30,
    order=5,
    sample_rate=fs
)

# Extract R-peaks and RR-intervals
peaks = pk.ecg.find_peaks(ecg_clean, sample_rate=fs)
rri = pk.ecg.compute_rr_intervals(peaks)
mask = pk.ecg.filter_rr_intervals(rri, sample_rate=fs)

# Compute HRV metrics
hrv_td = pk.hrv.compute_hrv_time(
    rr_intervals=rri[mask == 0],
    sample_rate=fs
)


In [ ]:
dataclasses.asdict(hrv_td)


In [ ]:
band_names = ["VLF", "LF", "HF", "VHF"]
bands = [(0.0033, 0.04), (0.04, 0.15), (0.15, 0.4), (0.4, 0.5)]
hrv_fd = pk.hrv.compute_hrv_frequency(peaks[mask == 0], rri[mask == 0], bands=bands, sample_rate=fs)


fig = go.Figure()
fig.add_trace(go.Bar(
    x=np.array([b.total_power for b in hrv_fd.bands])/hrv_fd.total_power,
    y=band_names,
    marker_color=[primary_color, secondary_color, tertiary_color, quaternary_color],
    orientation='h'
))

fig.update_xaxes(title_text="Normalized Power")
fig.update_layout(
    plot_bgcolor=bg_color,
    paper_bgcolor=bg_color,
    margin=dict(l=10, r=5, t=40, b=20),
    height=300,
)
fig.write_html(dst_path / f"hrv_frequency_power.html", include_plotlyjs='cdn', full_html=False)
fig.show()


In [ ]:
fs = 1000
tgt_rr = 12.6
rc_amp = 1.5
ab_amp = 1.0
dur_sec = 60

# Synthesize RC and AB band data
rc = rc_amp*pk.rsp.synthesize(
    duration=dur_sec,
    sample_rate=fs,
    respiratory_rate=tgt_rr
)
ab = ab_amp*pk.rsp.synthesize(
    duration=dur_sec,
    sample_rate=fs,
    respiratory_rate=tgt_rr
)

ts = np.arange(0, rc.size) / fs


In [ ]:
ts_metrics, dual_metrics = [], []
for i in range(0, rc.size - 10*fs, 1*fs):
    rc_win = rc[i:i+10*fs]
    ab_win = ab[i:i+10*fs]
    ts_metrics.append((i+5*fs)/fs)
    dual_metrics.append(pk.rsp.compute_dual_band_metrics(
        rc=rc_win,
        ab=ab_win,
        sample_rate=fs,
        pwr_threshold=0.9
    ))


In [ ]:
fig = make_subplots(rows=5, cols=1, shared_xaxes=True, vertical_spacing=0.05)

fig.add_trace(go.Scatter(
    x=ts,
    y=rc,
    line_color=primary_color,
    line_width=2,
    name='RC'
), row=1, col=1)
fig.add_trace(go.Scatter(
    x=ts,
    y=ab,
    line_color=secondary_color,
    line_width=2,
    name='AB'
), row=1, col=1)
fig.update_yaxes(title="Bands", row=1, col=1)

fig.add_trace(go.Scatter(
    x=ts_metrics,
    y=[metric.rc_rr for metric in dual_metrics],
    name='RC BPM',
    line_color=primary_color,
    line_width=2,

), row=2, col=1)
fig.add_trace(go.Scatter(
    x=ts_metrics,
    y=[metric.ab_rr for metric in dual_metrics],
    name='AB BPM',
    line_color=secondary_color,
    line_width=2,
), row=2, col=1)
fig.add_trace(go.Scatter(
    x=ts_metrics,
    y=[metric.vt_rr for metric in dual_metrics],
    name='VT BPM',
    line_color=tertiary_color,
    line_width=2,
), row=2, col=1)
fig.update_yaxes(title="BPM", row=2, col=1)

fig.add_trace(go.Scatter(
    x=ts_metrics,
    y=[metric.phase for metric in dual_metrics],
    name='Phase',
    line_width=2,
), row=3, col=1)
fig.update_yaxes(title="Phase", row=3, col=1)


fig.add_trace(go.Scatter(
    x=ts_metrics,
    y=[metric.lbi for metric in dual_metrics],
    name='LBI',
    line_width=2,
), row=4, col=1)
fig.update_yaxes(title="LBI", row=4, col=1)

fig.add_trace(go.Scatter(
    x=ts_metrics,
    y=[metric.rc_percent for metric in dual_metrics],
    name='%RC',
    line_width=2,
), row=5, col=1)
fig.update_yaxes(title="%RC", row=5, col=1)

fig.update_xaxes(title='Time (s)', range=[5, 54], row=5, col=1)

fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    title="",
    template="plotly_dark",
    plot_bgcolor=bg_color,
    paper_bgcolor=bg_color,
    margin=dict(l=10, r=5, t=60, b=20),
    height=500,
)

fig.write_html(dst_path / f"pk_rsp_dual_metrics.html", include_plotlyjs='cdn', full_html=False)
fig.show()
